# ESPAAN

In [1]:
!rm -r spaan
!git clone git://github.com/nicolagulmini/spaan
#!pip install Bio

Cloning into 'spaan'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 90 (delta 47), reused 46 (delta 18), pack-reused 0
Receiving objects: 100% (90/90), 731.64 KiB | 11.08 MiB/s, done.
Resolving deltas: 100% (47/47), done.


In [2]:
from spaan.utils import *
from spaan.Expanded_SPAAN_Model import *

In [3]:
from Bio import SeqIO
import numpy as np

adhesin_dataset = list(SeqIO.parse("spaan/data/original_adh_dataset.fasta", "fasta"))
not_adhesin_dataset = list(SeqIO.parse("spaan/data/original_negative_dataset.fasta", "fasta"))

In [4]:
# data processing

x, y = [], []
for protein in adhesin_dataset:
    x.append(np.array([aminoacids_frequencies(protein.seq),
                        multiplet_frequencies(protein.seq, 3),
                        multiplet_frequencies(protein.seq, 4),
                        multiplet_frequencies(protein.seq, 5),
                        dipeptide_frequencies(protein.seq),
                        charge_composition(protein.seq),
                        hydrophobic_composition(protein.seq)], dtype=object))
    y.append(1)

for protein in not_adhesin_dataset:
    x.append(np.asarray([aminoacids_frequencies(protein.seq),
                        multiplet_frequencies(protein.seq, 3),
                        multiplet_frequencies(protein.seq, 4),
                        multiplet_frequencies(protein.seq, 5),
                        dipeptide_frequencies(protein.seq),
                        charge_composition(protein.seq),
                        hydrophobic_composition(protein.seq)], dtype=object))
    y.append(0)

In [5]:
x = np.array(x, dtype=object)
y = np.array(y, dtype=np.float)

shuffler = np.random.permutation(len(x)) 
# shuffle
x = x[shuffler]
y = y[shuffler] 
# train, val, test division
train_len = int(len(x)/2)
x_train = x[:train_len]
y_train = y[:train_len]
x = x[train_len:]
y = y[train_len:]
val_len = int(len(x)/2)
x_val = x[:val_len]
y_val = y[:val_len]
x_test = x[val_len:]
y_test = y[val_len:]

In [6]:
# model instantiation
espaan_model = Expanded_SPAAN_Model()

In [ ]:
history = espaan_model.get_model().fit(
    x=x_train,
    y=y_train,
    batch_size=32, # default
    epochs=10, # start with 10 (the original paper has 10k epochs!)
    verbose=1,
    #validation_data=(x_val, y_val),
    shuffle=True    
    )

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(epochs), history[1])